In [0]:
%sh
apt update


In [0]:
%sh
apt-get install -y unoconv libreoffice

In [0]:
#List of dependencies
import os
import shutil
import copy
import pandas as pd
import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import ast
from bs4 import BeautifulSoup
from datetime import datetime
from docx import Document
from docx.oxml import parse_xml
import subprocess
import requests
import io
from docx.opc.constants import CONTENT_TYPE as CT
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls
from docx.shared import Inches
PATH = 'xxxx'
PATH1 = 'xxxx'
PATH2 = 'xxxx'
PATH2 = 'xxxx'


if not('IS_FIRST_NOTEBOOK_RUN' in locals()):
#     os.system("apt-get -f -y install xxxxx")
    IS_FIRST_NOTEBOOK_RUN = True


In [0]:
#####################################################################################
################################### ONE OFF SETUP ###################################
#####################################################################################
#Mounting the blob storage for dengue documents
storage_name="xxxx"
container_name="xxxx"
blobkey="xxxx"

blobconf = "fs.azure.account.key."+storage_name+".blob.core.windows.net"
if not any(mount.mountPoint == "/mnt/"+storage_name+ "/" +container_name for mount in dbutils.fs.mounts()):
    print('nothing mounted')
    try:
        dbutils.fs.mount(
          source = "wasbs://"+container_name+"@"+storage_name+".blob.core.windows.net",
          mount_point = "/mnt/"+storage_name+ "/" +container_name,
          extra_configs = {blobconf:blobkey})
    except Exception as e:
        print("already mounted. Try to unmount first")

PATH = '/dbfs/mnt/'+storage_name+'/'+container_name +'/'


In [0]:
# Email body 
email_body = '''
<p>Dear our valuable customers,</p><br>
<p>Congratulations, your application for purchasing a vehicle insurance product from Zurich Asuransi Indonesia has been approved.</p><br>
<p>Your vehicle’s license plate: {text}</p><br>
<p>We attach the results of the assessment that has been verified.</p>
<p>Should you need further information on your vehicle, please contact your Zurich agent.</p><br>
<p>PT Zurich Asuransi Indonesia Tbk is licensed and registered by the Financial Services Authority (OJK).</p>
'''




In [0]:
def replace_placeholders(doc, replacements):
    page_number = 1
    for paragraph in doc.paragraphs:
        if page_number <= 2:
            for key, value in replacements.items():
                if key in paragraph.text:
                    paragraph.text = paragraph.text.replace(key, value)
                    
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for key, value in replacements.items():
                    if key in cell.text:
                        cell.text = cell.text.replace(key, value)

    return doc

def fill_images_in_table(doc, image_folder):
    image_files = os.listdir(image_folder)
    table = doc.tables[0]  # Get the first table in the document

    row_index = 2  # Index of the third row (0-based indexing)
    cell_index = 0  # Index of the first cell in the row (0-based indexing)

    image_counter = 0  # Counter to keep track of the number of inserted images

    while image_counter < len(image_files):
        if cell_index >= len(table.columns):
            # Move to the next row if the current row is filled
            row_index += 1
            cell_index = 0

        if row_index >= len(table.rows):
            # Break if we have exhausted all rows in the table
            break

        cell = table.cell(row_index, cell_index)
        image_path = os.path.join(image_folder, image_files[image_counter])
        cell.text = ""

        # Add the image to the cell
        cell_paragraph = cell.paragraphs[0]
        run = cell_paragraph.add_run()
        run.add_picture(image_path, width=Inches(1.5), height=Inches(1.5))

        image_counter += 1
        cell_index += 1

        if cell_index % 4 == 0:
            # Move to the next row after inserting four images in the current row
            row_index += 1
            cell_index = 0

    return doc


def save_docx_to_file(doc, file_path):
    buffer = io.BytesIO()
    doc.save(buffer)
    buffer.seek(0)
    with open(file_path, "wb") as f:
        f.write(buffer.read())

def convert_to_pdf(input_path, output_path):
        subprocess.run(['unoconv', '-f', 'pdf', '-o', output_path, input_path], check=True)
        #subprocess.run(['unoconv', '-f', 'pdf', '-o', output_path, input_path])

In [0]:
df_camcom_review_completed = pd.read_csv('xxxx'+ 'df_camcom_review_completed.csv')


In [0]:
# every time a new blob is uploaded or modified
# read all the files 
#filename = dbutils.widgets.get('filename')
#camcom-sftp/camcom-data/zurichidsa_camcom_id.csv

file_list = os.listdir(PATH1 + "camcom-sftp/camcom-data")
import pickle
with open(PATH +'file_list_before.pickle', 'rb') as file:
    file_list_before = pickle.load(file)

for i in file_list:
    if i not in file_list_before:
        print(i)
        if '_review_completed.csv' in str(i):
            file_list_before.append(i)
            inspection_id = i.split('_')[0]
            print(inspection_id)
    
        # if inspection_id:
            df_merged = df_camcom_review_completed[df_camcom_review_completed['Inspection ID'] == inspection_id].reset_index()
            df_merged['Type of Damages'] = df_merged['Type of Damages'].apply(lambda x: '' if x == '-' else x)

            df = df_merged[['Damaged Part','Type of Damages','Action']].copy()
            grouped = df.groupby(['Damaged Part', 'Type of Damages'])['Action'].apply(list).reset_index()
            print(grouped)
            repair_list = []
            replace_list = []

            for _, row in df.iterrows():
                damaged_part = row['Damaged Part']
                damage_type = row['Type of Damages']
                action = row['Action']

                action_text = f'{damaged_part} / {damage_type}'

                if action == 'Repair':
                    repair_list.append(action_text)
                
                if action == 'Replace':
                    replace_list.append(action_text)

            repair_list = repair_list
            replace_list = replace_list

            repair_list = '\n'.join(repair_list)
            replace_list = '\n'.join(replace_list)
            print('repair_list:', repair_list)
            print('replace_list:', replace_list)

            template_path = "xxxx/template/ZSurvey Assessment Result.docx"
            image_folder = "xxxxsftp/images/" + inspection_id
            print(image_folder)
            PATH = "xxxx/template/"
            doc = Document(template_path)

            temp_df = pd.read_csv("xxxx" + inspection_id +'_review_completed.csv')
            reviewd_date = temp_df['Review Completed Date'][0]

            file_name_doc = PATH2 + temp_df['Licence Plate'][0] + ".docx"
            file_name_pdf = PATH2 + temp_df['Licence Plate'][0] + ".pdf"
            replacements = {
            "{{placeholder1}}": temp_df['Licence Plate'][0],
            "{{placeholder2}}": reviewd_date,
            "{{placeholder3}}": repair_list,
            "{{placeholder4}}": replace_list
            }

            replace_placeholders(doc, replacements)
            fill_images_in_table(doc,image_folder)

            save_docx_to_file(doc, file_name_doc)
            convert_to_pdf(file_name_doc, file_name_pdf)
            email_body = email_body.replace("{text}", temp_df['Licence Plate'][0])

            text_file = open(PATH2 + temp_df['Licence Plate'][0] + ".txt", "w")
            n = text_file.write(email_body)
            text_file.close()
        # else:
        #     pass

        else:
            file_list_before.append(i)
    


In [0]:
import pickle
with open(PATH +'file_list_before.pickle', 'wb') as file:
    pickle.dump(file_list_before, file)

In [0]:
# file_list_before = os.listdir(PATH1+'camcom-sftp/camcom-data')